# Линейная регрессия
## Одномерный случай

Имеем линейную модель $a(x) = w * x + b$, тогда функция потерь с MSE будет следующей: $L = 1 / N * \sum_{i=1}^{N} (y_i - wx_i - b)^2$

Производная по w будет следующей: $\frac{\partial L}{\partial w} = -2 / N * \sum_1^N x_i(y_i - wx_i - b)$

А производная по b: $\frac{\partial L}{\partial b} = - 2 / N * \sum_1^N (y_i - wx_i - b)$

## Многомерный случай

Путь $x_i$ - вектор, такой что $x_i = (x_{i, 1}, x_{i, 2}, ..., x_{i, M})$, тогда модель принимает вид $a(x) = (w, x) + b$, где $(w, x)$ - скалярное произведение векторов.

Тогда $L = 1/N * \sum_1^N (y_i - (w, x_i) - b)^2$

Частные производные по параметрам:

$\frac{\partial L}{\partial b} = -2/N * \sum_1^N (y_i - (w, x_i) - b)$

$\frac{\partial L}{\partial w_1} = -2/N * \sum_1^N x_{i,1}(y_i - (w, x_i) - b)$

$\frac{\partial L}{\partial w_M} = -2/N * \sum_1^N x_{i,M}(y_i - (w, x_i) - b)$

## Укорение вычислений

Можно предсохранить вычисления для $(y_i - (w, x_i) - b)$, но пока не ясно, насколько это имеет смысл, т.к. в np будет производиться скалярное произведение всех x на $(y_i - (w, x_i) - b)$, то есть частные производные не будут считать по одной.

## Точка минимума

Рассмотрим $f'(x) = 0$. Производная (тангенс угла наклона) равна нулю, а значит есть переход функции с + на - или с - на +.

$L'(x) = -2k/N \sum_1^N (y_i - a(x_i)) = 0$; $k = 0$ или $y_i = a(x_i)$

Для многомерного случая используем $\nabla f = 0$

In [38]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

np.random.seed(0)

In [5]:
def train_linear_regression(x, y, batch_size=None):
    lr = 0.01
    num_epochs = 1000

    all_data_len = len(y)
    if batch_size is None:
        batch_size = all_data_len
    w_pred = np.random.randn(x.shape[1])
    b_pred = np.random.randn()

    for _ in range(num_epochs):
        for i in range(0, all_data_len, batch_size):
            x_batch = x[i:i + batch_size]
            y_batch = y[i:i + batch_size]
            y_pred = np.dot(x_batch, w_pred) + b_pred
            dw = (-2 / batch_size) * np.dot(x_batch.T, (y_batch - y_pred))
            db = (-2 / batch_size) * np.sum(y_batch - y_pred)
            w_pred -= lr * dw
            b_pred -= lr * db

    return w_pred, b_pred

def get_data(n=10, data_size=1024):
    w_true = np.random.randn(n) * 3
    b_true = np.random.rand(1) * 3

    noise = np.random.randn(data_size)
    x = np.random.rand(data_size, n)

    y = (w_true[None] * x).sum(1) + b_true + noise

    return w_true, b_true, x, y

w_true, b_true, x, y = get_data()

w_pred, b_pred = train_linear_regression(x, y)

print(w_pred, w_true)
print(b_pred, b_true)

[ 2.61569777  1.02560237  1.34191146  3.86903426  0.57096603  3.3194523
 -1.07382633 -0.34782617 -0.13654499 -2.19243998] [ 3.14227592  0.89140512  1.78540228  4.83497825  0.42737684  3.8937452
 -1.44609461 -0.48534022  0.11902165 -2.67842662]
1.1781282395297192 [0.42641612]


In [6]:
w_true, b_true, x, y = get_data(n=1, data_size=1024)
w_pred, b_pred = train_linear_regression(x, y, batch_size=64)
print(x.shape, y.shape)

(1024, 1) (1024,)


In [23]:
def calc_pred(w_pred, b_pred, x):
    return np.dot(x, w_pred) + b_pred

In [35]:
lin_space = np.linspace(0, 1, 100)
lin_space.shape
lin_space = lin_space.reshape(lin_space.shape[0], 1)
res = calc_pred(w_pred=w_pred, b_pred=b_pred, x=lin_space)
res.shape

(100,)

In [36]:
calc_pred(w_pred, b_pred, lin_space).shape

(100,)

In [37]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=lin_space.reshape(lin_space.shape[0]), y=calc_pred(w_pred, b_pred, lin_space), mode="lines", name="Predicted model"))
fig.add_trace(go.Scatter(x=x.reshape(x.shape[0]), y=y, mode="markers", name="True value"))
fig.show()

In [39]:
sk_model = LinearRegression()
sk_model.fit(x, y)

LinearRegression()

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=lin_space.reshape(lin_space.shape[0]), y=sk_model.predict(lin_space), mode="lines", name="Sklearn predicted model"))
fig.add_trace(go.Scatter(x=lin_space.reshape(lin_space.shape[0]), y=calc_pred(w_pred, b_pred, lin_space), mode="lines", name="Predicted model"))
fig.add_trace(go.Scatter(x=x.reshape(x.shape[0]), y=y, mode="markers", name="True value"))
fig.show()